# Hoti

In [ ]:
from pylab import *
mpl.rcParams.update({'font.size':18})

In [ ]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

if not os.path.exists("images/video_images"):
    os.mkdir("images/video_images")

if not os.path.exists("images/video_images3"):
    os.mkdir("images/video_images3")

# Modelo finito 

In [ ]:
x,y,f = np.loadtxt("LatticeSierpinski_18x18.xyz",usecols=(1,2,4),skiprows=2,unpack=True)

In [ ]:
r = 1.1
Dic = {}
natoms = int( len(x) )
for n in range(natoms):
    a = sqrt( (x[n]-x)*(x[n]-x) + (y[n]-y)*(y[n]-y) ) < r
    vecinos = [i for i, x in enumerate(a) if x and i!=n]
    Dic[n] = vecinos

In [ ]:
def Hoti_Hamiltonian(γ, λ):
    H = np.zeros((natoms,natoms), dtype = 'complex')
    Hop = {}
    a = exp(-1J*pi)
    δ = 1e-8
    ϵ = 0.1
    for n in Dic.keys():
        vecinos = Dic[n]
        hop = []
        for v in vecinos:
            f0 = f[n]
            f1 = f[v]
            if f0 == f1:
                fase = γ 
            else: 
                fase = λ 
            H[n,v] = fase + fase*ϵ*rand()
            H[v,n] = fase + fase*ϵ*rand()
            
            
        if n%2==0:
            H[n,n] = δ
        else:
            H[n,n] = -δ
            
            hop.append(fase)
        Hop[n] = hop

    f0 = 0
    f1 = 0
    fase = 1
    for n in Dic.keys():
        vecinos = Dic[n]
        c = 0
        for v in vecinos:
            posx     = x[n]
            posy     = y[n]
            f0       = f[n]
            f1       = f[v] 
            if f0 == f1:
                fase = γ
            else: 
                fase = λ

            if ( posx%2==0 and posy%2==0  ):
                if ( x[v] > x[n] ):
                    H[(n,v)] = a*fase + fase*ϵ*rand()
                    H[(v,n)] = a*fase + fase*ϵ*rand()
            elif(posx%2==1 and posy%2==1):
                if ( x[v] > x[n] ):
                    H[(n,v)] = a*fase + fase*ϵ*rand()
                    H[(v,n)] = a*fase + fase*ϵ*rand()
            else:
                pass
    return H

In [ ]:
H0 = Hoti_Hamiltonian(1.0, 5.0)
H1 = Hoti_Hamiltonian(1.0, 1.0)
H2 = Hoti_Hamiltonian(5.0, 1.0)

np.savetxt('halmitonian0.txt',H0)
np.savetxt('halmitonian1.txt',H1)
np.savetxt('halmitonian2.txt',H2)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=3,
                    subplot_titles=("γ/λ < 1 ","γ/λ = 1", "γ/λ > 1"))

fig.add_trace(
    go.Heatmap(z=H0.real,coloraxis = "coloraxis"),
    row=1, col=1)

fig.add_trace(
    go.Heatmap(z=H1.real,coloraxis = "coloraxis"),
    row=1, col=2)

fig.add_trace(
    go.Heatmap(z=H2.real,coloraxis = "coloraxis"),
    row=1, col=3)

fig.update_xaxes(showgrid=False,showticklabels=False)
fig.update_yaxes(showgrid=False,showticklabels=False)
fig.update_layout(height=400, width=900, showlegend=False, title_text="", coloraxis={'colorscale':'RdBu'})
fig.show()
fig.write_image("images/Hamiltonian_cero.pdf")

In [ ]:
Ek0, U0 = eigh(H0)
Ek0 = np.real(Ek0)

Ek1, U1 = eigh(H1)
Ek1 = np.real(Ek1)

Ek2, U2 = eigh(H2)
Ek2 = np.real(Ek2)


savetxt("Energias0.txt",Ek0,fmt = '%s', comments = '')
savetxt("Estados0.txt",U0,fmt = '%s', comments = '')

savetxt("Energias1.txt",Ek1,fmt = '%s', comments = '')
savetxt("Estados1.txt",U1,fmt = '%s', comments = '')

savetxt("Energias2.txt",Ek2,fmt = '%s', comments = '')
savetxt("Estados2.txt",U2,fmt = '%s', comments = '')

In [ ]:
Ek0 = loadtxt("Energias0.txt", unpack=True)
U0  = loadtxt("Estados0.txt", dtype=complex, unpack=True)

Ek1 = loadtxt("Energias1.txt", unpack=True)
U1  = loadtxt("Estados1.txt", dtype=complex, unpack=True)

Ek2 = loadtxt("Energias2.txt", unpack=True)
U2  = loadtxt("Estados2.txt", dtype=complex, unpack=True)

In [ ]:
fig = make_subplots(rows=1, cols=3,
                   subplot_titles=("γ/λ < 1 ","γ/λ = 1", "γ/λ > 1"))

bins = 100

fig.add_trace(
    go.Histogram(x=Ek0, nbinsx=bins), row=1, col=1)

fig.add_trace(
    go.Histogram(x=Ek1, nbinsx=bins), row=1, col=2)

fig.add_trace(
    go.Histogram(x=Ek2, nbinsx=bins), row=1, col=3)

        
fig.update_xaxes(title_text="E")
fig.update_yaxes(title_text="#E")
fig.update_layout(height=400, width=900, title_text="", showlegend=False)
fig.show()
fig.write_image("images/Dos_cero.png")

In [ ]:
def wave_function(Ek,U):
    indexsort = argsort(Ek)
    Est = [i for i,val in enumerate(Ek) if abs(val) < 0.2]
    ek = np.mean(Ek[Est])
    s = [np.real(U[i]*U.conjugate()[i]) for i in Est]
    S = np.zeros(len(s[0]))
    for i in range(len(s)):
        S += s[i] 
    S = S/len(s)
    Est = Est
    return S,Est

In [ ]:
W0,Est = wave_function(Ek0,U0)

In [ ]:
def density_of_states(Ek,U):
    indexsort = argsort(Ek)
    #Est = [i for i,val in enumerate(Ek) if abs(val) < 0.2]
    ek = np.mean(Ek[Est])
    s = [np.real(U[i]*U.conjugate()[i]) for i in Est]
    S = np.zeros(len(s[0]))
    for i in range(len(s)):
        S += s[i] 
    S = S/len(s)
    #print(Est)
    return S

In [ ]:
def density_of_states_array(Ek,U):
    indexsort = argsort(Ek)
    #Est = [i for i,val in enumerate(Ek) if abs(val) < 0.2]
    ek = np.mean(Ek[Est])
    s = [np.real(U[i]*U.conjugate()[i]) for i in Est]
    S = np.zeros(len(s[0]))
    for i in range(len(s)):
        S += s[i] 
    S = S/len(s)
    #print(Est)
    return s

In [ ]:
S0 = density_of_states(Ek0,U0)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x+10, y=y+10,
                    mode='markers',
                    name='markers'))
fig.add_trace(go.Scatter(
    x=x+10,
    y=y+10,
    mode='markers',
    marker=dict(size=S0*200)))
fig.update_xaxes(showgrid=False,showticklabels=False)
fig.update_yaxes(showgrid=False,showticklabels=False)
fig.update_layout(showlegend=False,height=600, width=600, title_text="")
fig.show()
fig.write_image("images/Proyection_cero.png")

In [ ]:
S0 = density_of_states(Ek0,U0)
wave = reshape(S0, (18,18))

In [ ]:
for i in range(18):
    for j in range(18):
        n = 1
        if i<=8 and j <=8:
            n = -1
        elif i>8 and j >8:
            n = -1
        else :
            n = 1
        
        wave[i,j] = wave[i,j]*n 

In [ ]:

fig = go.Figure(data=[go.Surface(z=wave, colorscale='RdBu')])

fig.update_layout(title='', autosize=False,
                  width=700, height=700,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
E_γλ = [] #run onece
for γ in linspace(-2.5,2.5,51):
    H = Hoti_Hamiltonian(γ,1)
    Ek, U = eigh(H)
    Ek = np.real(Ek)
    indexsort = argsort(Ek)
    E_γλ.append(Ek[indexsort])
E = array(E_γλ)    
savetxt("param_bands.txt",E,fmt = '%s', comments = '')

In [ ]:
E = loadtxt("param_bands.txt",unpack=True)
n = len(E.T[0])
posx = linspace(-2.5,2.5,51)
fig = go.Figure()
for i in range(n):
    if i in Est:
        fig.add_trace(go.Scatter(x=posx,y=E[i], marker_color="red")) 
    else:
        fig.add_trace(go.Scatter(x=posx,y=E[i], marker_color="blue")) 
        
fig.update_xaxes(showgrid=False,showticklabels=True, title_text="γ/λ")
fig.update_yaxes(showgrid=False,showticklabels=True, title_text="E")
fig.update_layout(showlegend=False, height=600, width=600, title_text="")
fig.show()
fig.write_image("images/Bands_ce.png")

In [ ]:
n = len(E.T[0])
posx = linspace(-2.5,2.5,51)
fig = go.Figure()
for i in range(n):
    if i in Est:
        fig.add_trace(go.Scatter(x=posx,y=log(abs(E[i])), marker_color="red")) 
    else:
        fig.add_trace(go.Scatter(x=posx,y=log(abs(E[i])), marker_color="blue")) 
        
fig.update_xaxes(showgrid=False,showticklabels=True, title_text="γ/λ")
fig.update_yaxes(showgrid=False,showticklabels=True, title_text="log |E| ")
fig.update_layout(showlegend=False, height=600, width=600, title_text="")
fig.show()
fig.write_image("images/Bands_ce_log.png")

# Modelo Infinito

In [ ]:
def SSH(kx,ky,γ,λ):
    δ = 1e-5
    H = np.array([[δ,γ+λ*exp(-1J*kx),0,γ-λ*exp(1J*ky)],
                  [γ+λ*exp(1J*kx),-δ,-γ+λ*exp(1J*ky),0],
                  [0,-γ+λ*exp(-1J*ky),-δ,γ+λ*exp(1J*kx)],
                  [γ-λ*exp(-1J*ky),0,γ+λ*exp(-1J*kx),δ]])
    return H

In [ ]:
Kx, Ky =  linspace(-pi,pi,50), linspace(-pi,pi,50)
kxg,kyg = [],[]
Edat = []
for dat in [[0.0,5.0],[1.0,1.0],[5.0,1.0]]:
    E = []
    for kx in Kx:
        for ky in Ky:
            H = SSH(kx,ky,γ=dat[0],λ=dat[1])
            Ek,vk = eigh(H)
            E.append(Ek)
            
    E = array(E)
    Edat.append(E)

In [ ]:
fig = go.Figure()
for i in range(len(Edat[0].T)):
    M = reshape(Edat[0].T[i],(50,50))
    fig.add_trace(go.Surface(z=M,  showscale=False, opacity=0.9))
    fig.update_layout(showlegend=False, height=600, width=600, title_text="γ/λ < 1")
    
fig.show()
fig.write_image("images/bands_surface1.png")

In [ ]:
fig = go.Figure()
for i in range(len(Edat[1].T)):
    M = reshape(Edat[1].T[i],(50,50))
    fig.add_trace(go.Surface(z=M,  showscale=False, opacity=0.9))
    fig.update_layout(showlegend=False, height=600, width=600, title_text="γ/λ = 1")

fig.show()
fig.write_image("images/bands_surface2.png")

In [ ]:
fig = go.Figure()
for i in range(len(Edat[2].T)):
    M = reshape(Edat[2].T[i],(50,50))
    fig.add_trace(go.Surface(z=M,  showscale=False, opacity=0.9))
    fig.update_layout(showlegend=False,height=600, width=600, title_text="γ/λ > 1")
    
fig.show()
fig.write_image("images/bands_surface3.png")

# High Order Topological Pump

In [ ]:
δ = 1e-3
C   = 1.0     # cm^-1
κ   = 1    # μm^-1
L   = 1   # μm
A   = 0.5     # μm
γ, λ = 1.0,1.0
fig = go.Figure()


def func_cext(θ,A,λ):
    return  λ*exp(-κ*(L+A*cos(θ)))

def func_cint(θ,A,γ):
    return γ*exp(-κ*(L-A*cos(θ)))


In [ ]:
def Hoti_Hamiltonian_transport(A, θ, γ):
    H = np.zeros((natoms,natoms), dtype = 'complex')
    a = exp(-1J*pi)
    δ = 1e-5
    λ = 1.0
    ϵ = 0.1
    for n in Dic.keys():
        vecinos = Dic[n]
        for v in vecinos:
            f0 = f[n]
            f1 = f[v]
            if f0 == f1:
                fase = func_cint(θ,A,γ) 
            else: 
                fase = func_cext(θ,A,λ)
            H[n,v] = fase + fase*ϵ*rand()
            H[v,n] = fase + fase*ϵ*rand()
            
        if n%2==0:
            H[n,n] = δ*sin(θ)# + 1e-8*random()
        else:
            H[n,n] = -δ*sin(θ)# - 1e-8*random()
            

    f0 = 0
    f1 = 0
    fase = 1
    for n in Dic.keys():
        vecinos = Dic[n]
        c = 0
        for v in vecinos:
            posx     = x[n]
            posy     = y[n]
            f0       = f[n]
            f1       = f[v] 
            if f0 == f1:
                fase = func_cint(θ,A,γ)
            else: 
                fase = func_cext(θ,A,λ)

            if ( posx%2==0 and posy%2==0  ):
                if ( x[v] > x[n] ):
                    H[(n,v)] = a*fase + fase*ϵ*rand()
                    H[(v,n)] = a*fase + fase*ϵ*rand()
            elif(posx%2==1 and posy%2==1):
                if ( x[v] > x[n] ):
                    H[(n,v)] = a*fase + fase*ϵ*rand()
                    H[(v,n)] = a*fase + fase*ϵ*rand()
            else:
                pass
    return H

In [ ]:
A  = 0.5
H0 = Hoti_Hamiltonian_transport(A,-pi,1)
H1 = Hoti_Hamiltonian_transport(A,0,1)
H2 = Hoti_Hamiltonian_transport(A,pi,1)

Ek0, U0 = eigh(H0)
Ek0 = np.real(Ek0)

Ek1, U1 = eigh(H1)
Ek1 = np.real(Ek1)

Ek2, U2 = eigh(H2)
Ek2 = np.real(Ek2)

savetxt("E0_pump.txt",Ek0,fmt = '%s', comments = '')
savetxt("Est0_pump.txt",U0,fmt = '%s', comments = '')

savetxt("E1_pump.txt",Ek1,fmt = '%s', comments = '')
savetxt("Est1_pump.txt",U1,fmt = '%s', comments = '')

savetxt("E2_pump.txt",Ek2,fmt = '%s', comments = '')
savetxt("Est2_pump.txt",U2,fmt = '%s', comments = '')

In [ ]:
Ek0 = loadtxt("E0_pump.txt", unpack=True)
U0  = loadtxt("Est0_pump.txt", dtype=complex, unpack=True)

Ek1 = loadtxt("E1_pump.txt", unpack=True)
U1  = loadtxt("Est1_pump.txt", dtype=complex, unpack=True)

Ek2 = loadtxt("E2_pump.txt", unpack=True)
U2  = loadtxt("Est2_pump.txt", dtype=complex, unpack=True)

In [ ]:
fig = make_subplots(rows=1, cols=3,
                   subplot_titles=("θ = -π ","θ = 0", "θ = π"))

bins = 101

fig.add_trace(
    go.Histogram(x=Ek0, nbinsx=bins), row=1, col=1)

fig.add_trace(
    go.Histogram(x=Ek1, nbinsx=bins), row=1, col=2)

fig.add_trace(
    go.Histogram(x=Ek2, nbinsx=bins), row=1, col=3)
        
fig.update_xaxes(title_text="E")
fig.update_yaxes(title_text="#E")
fig.update_layout(showlegend=False, height=400, width=900, title_text="")
fig.write_image("images/Dos_pump.png")

fig.show()

In [ ]:
fig = go.Figure()
λ = 1.0
A = 0.5

θ = linspace(-pi,pi,100)
fig.add_trace(go.Scatter(x=func_cint(θ,A,0)-func_cext(θ,A,λ),y=δ*sin(θ),name = "γ/λ={}".format(round(0,3)), mode='lines')) 
fig.add_trace(go.Scatter(x=func_cint(θ,A,0.36)-func_cext(θ,A,λ),y=δ*sin(θ),name = "γ/λ={}".format(round(0.36,3)), mode='lines')) 
fig.add_trace(go.Scatter(x=func_cint(θ,A,0.606)-func_cext(θ,A,λ),y=δ*sin(θ),name = "γ/λ={}".format(round(0.606,3)), mode='lines')) 
fig.add_trace(go.Scatter(x=func_cint(θ,A,1)-func_cext(θ,A,λ),y=δ*sin(θ),name = "γ/λ={}".format(round(1,3)), mode='lines')) 

    
fig.add_trace(go.Scatter(x=[-0.7,0.5],y=[0,0],name = "", mode='lines', marker_color="black",line=dict(width=1)))
fig.add_trace(go.Scatter(x=[0,0],y=[-0.0012,0.0012],name = "", mode='lines', marker_color="black",line=dict(width=1)))


fig.update_xaxes(title_text='cint-cext', showgrid=True)
fig.update_yaxes(title_text='δsenθ', showgrid=True)
fig.update_layout(height=600, width=800, title_text="")
fig.write_image("images/param_pump.png")
    

fig.show()

In [ ]:
fig = make_subplots(rows=10, cols=3,
                   subplot_titles=("θ = -π ","θ = 0", "θ = π"))

bins = 1001

n = 1

for A in linspace(0,0.6,10):
    H0 = Hoti_Hamiltonian_transport(A,-pi,0.61)
    H1 = Hoti_Hamiltonian_transport(A,0,0.61)
    H2 = Hoti_Hamiltonian_transport(A,pi,0.61)


    Ek0, U0 = eigh(H0)
    Ek0 = np.real(Ek0)

    Ek1, U1 = eigh(H1)
    Ek1 = np.real(Ek1)

    Ek2, U2 = eigh(H2)
    Ek2 = np.real(Ek2)


    fig.add_trace(
        go.Histogram(x=Ek0, nbinsx=bins), row=n, col=1)

    fig.add_trace(
        go.Histogram(x=Ek1, nbinsx=bins), row=n, col=2)

    fig.add_trace(
        go.Histogram(x=Ek2, nbinsx=bins), row=n, col=3)

    fig.update_layout(showlegend=False,height=6*400, width=900, title_text="")
    n+=1    
    #print(A)
fig.update_xaxes(title_text="E")
fig.write_image("images/Dos_pump_paramA.png")
savetxt("Dos_pump_paramA.txt",linspace(0,0.6,10),fmt = '%s', comments = '')

fig.show()

In [ ]:
index = argsort(S0)[-4:]

step = 51
γs = linspace(0,5,step)
θs = linspace(-pi,pi,step)

M = zeros([step,step])


for i in range(step):
    for j in range(step):
        H = Hoti_Hamiltonian_transport(A,θs[i],γs[j])
        Ek, U = eigh(H)
        Ek = np.real(Ek)
        S = density_of_states(Ek.T,U.T)
        M[i,j]  =  sum(S[index])/4
        
savetxt("param_pump_energy.txt",M,fmt = '%s', comments = '')


In [ ]:
M = loadtxt("param_pump_energy.txt",unpack=True)
fig = go.Figure(go.Heatmap(x=θs,y=γs,z=log(M), colorscale='RdBu'))
fig.update_layout(height=600, width=600, title_text="")
#fig.add_trace(go.Scatter(x=[0.133,0.133],y=[-pi,pi],name = "", mode='lines', marker_color="black",line=dict(width=1)))
#fig.add_trace(go.Scatter(x=[0.5,0.5],y=[-pi,pi],name = "", mode='lines', marker_color="black",line=dict(width=1)))
fig.update_xaxes(title_text='θ')
fig.update_yaxes(title_text='γ/λ')
fig.write_image("images/param_pump_energy.png")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(x=θs,y=γs,z=log(M), colorscale='RdBu')])

fig.update_layout(title='', autosize=False,
                  width=700, height=700,
                  margin=dict(l=65, r=10, b=65, t=70))

fig.show()

In [ ]:
step = 51
γs = linspace(0,5,step)
θs = linspace(-pi,pi,step)

M = zeros([step,step])


for i in range(step):
    for j in range(step):
        H = Hoti_Hamiltonian_transport(0.5,θs[i],γs[j])
        Ek, U = eigh(H)
        Ek = np.real(Ek)
        M[i,j]  = min(abs(Ek))
savetxt("param_pump_Density.txt",M,fmt = '%s', comments = '')          

In [ ]:
M = loadtxt("param_pump_Density.txt",unpack=True)
fig = go.Figure(go.Heatmap(x=θs,y=γs,z=M,colorscale='RdBu'))
#fig.add_trace(go.Scatter(x=[0.133,0.133],y=[-pi,pi],name = "", mode='lines', marker_color="white",line=dict(width=1)))
#fig.add_trace(go.Scatter(x=[0.5,0.5],y=[-pi,pi],name = "", mode='lines', marker_color="white",line=dict(width=1)))
fig.update_layout(height=600, width=600, title_text="")
fig.update_xaxes(title_text='γ/λ')
fig.update_yaxes(title_text='θ')
fig.write_image("images/param_pump_Density.png")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(x=γs,y=θs,z=M, colorscale='RdBu')])

fig.update_layout(title='', autosize=False,
                  width=700, height=700,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
E_θ = []
posx = []
for θ in linspace(-pi,pi,50):
    H = Hoti_Hamiltonian_transport(0.5,θ,0.6)
    Ek, U = eigh(H)
    Ek = np.real(Ek)
    indexsort = argsort(Ek)
    E_θ.append(Ek[indexsort])
    posx.append(θ)
E = array(E_θ)    
savetxt("param_pump_A=0.5.txt",E,fmt = '%s', comments = '')

n = len(E[0])
fig = go.Figure()
for i in range(n):
    if i in Est:
        fig.add_trace(go.Scatter(x=posx,y=E.T[i], marker_color="red")) 
    else:
        fig.add_trace(go.Scatter(x=posx,y=E.T[i], marker_color="blue")) 
        
fig.update_xaxes(title_text='θ')
fig.update_yaxes(title_text='E')
fig.update_layout(height=600, width=600, title_text="",showlegend=False)
fig.write_image("images/param_pump_A=0.5.png")

fig.show()

In [ ]:
S0 = density_of_states(Ek0.T,U0.T)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='markers',
                    name='markers',
                    marker_color='blue'    ))
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    marker=dict(size=S0*70/max(S0)),
    marker_color='red'))
fig.update_xaxes(showgrid=False,showticklabels=False)
fig.update_yaxes(showgrid=False,showticklabels=False)
fig.update_layout(showlegend=False,height=600, width=600, title_text="")
fig.write_image("images/poyection_4cero.png")

fig.show()

In [ ]:
S_array = []
Pi_array = linspace(-pi,pi,51)
for θ in Pi_array:
    H = Hoti_Hamiltonian_transport(0.5,θ,0.6)
    Ek, U = eigh(H)
    Ek = np.real(Ek)
    S = density_of_states(Ek.T,U.T) 
    S_array.append(S)

In [ ]:
fig = go.Figure(
    
    data = [go.Scatter(x=x, y=y,
                    mode='markers',
                    name='markers',
                    marker=dict(color="blue", size=5)),
            go.Scatter(x=x, y=y,
                    mode='markers',
                    name='markers',
                    marker=dict(color="blue", size=1))
           ],
    
    layout=go.Layout(
        xaxis=dict(range=[min(x)-3, max(x)+3], autorange=False, zeroline=False,showgrid=False,showticklabels=False),
        yaxis=dict(range=[min(y)-3, max(y)+3], autorange=False, zeroline=False,showgrid=False,showticklabels=False),
        title_text="Evolution", hovermode="closest",width=600, height=600, 
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    
    frames=[go.Frame(
        data=[go.Scatter(
            x=x,
            y=y,
            mode="markers",
            marker=dict(color="red", size=S_array[k]*600)),
             ])

        for k in range(len(S_array))]
        
)

text = arange(len(S_array))
for i, frame in enumerate(fig.frames):
    frame.layout.title = "pi = {}, frame: {}".format(Pi_array[i],text[i])
    
fig.write_image("images/animated.jpg")

fig.show()

In [ ]:
import plotly.figure_factory as ff
Pi_array = linspace(-pi,pi,51)

H0 = Hoti_Hamiltonian_transport(0.5,-pi,0.6)
Ek0, U0 = eigh(H0)
Ek0 = np.real(Ek0)
s0 = density_of_states(Ek0.T,U0.T)
n = 0
for θ in Pi_array[1:]:
    H1 = Hoti_Hamiltonian_transport(0.5,θ,0.6)
    Ek1, U1 = eigh(H1)
    Ek1 = np.real(Ek1)
    s1 = density_of_states(Ek1.T,U1.T)
    
    Cx = [] 
    Cy = []
    Cu = [] 
    Cv = []
    for z in np.unique(f):
        find = [j for j,val in enumerate(f) if val == z]
        X = [x[i] for i in find]
        Y = [y[i] for i in find]
        S0 = [s0[i] for i in find]
        S1 = [s1[i] for i in find]
        xmax = np.amax(X)
        xmin = np.amin(X)
        ymax = np.amax(Y)
        ymin = np.amin(Y)
        rx = (xmax + xmin)/2
        ry = (ymax + ymin)/2
        Jx = vdot(subtract(S1,S0),X-rx)
        Jy = vdot(subtract(S1,S0),Y-ry)
        Cx.append(rx)
        Cy.append(ry)
        Cu.append(2000*Jx)
        Cv.append(2000*Jy)
    fig = go.Figure()
    fig = ff.create_quiver(Cx, Cy, Cu, Cv,
                       scale=1,
                       arrow_scale=.2,
                       line_width=2)
    fig.add_trace(go.Scatter(x=x, y=y,
                        mode='markers',
                        name='markers',
                        marker=dict(size=1, color='green'))) 
    fig.add_trace(go.Scatter(x=Cx, y=Cy,
                        mode='markers',
                        name='markers',
                        marker=dict(size=2, color='blue')))
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(size=s1*400, color='red')))
    
    fig.update_xaxes(showgrid=False,showticklabels=False,range=[min(x)-3, max(x)+3])
    fig.update_yaxes(showgrid=False,showticklabels=False,range=[min(y)-3, max(y)+3])
    fig.update_layout(showlegend=False, height=600, width=600, title_text="")
    fig.update_layout(showlegend=False, height=600, width=600, 
                      title_text="θ =  {}".format((pi*n/51)-pi))
    fig.write_image("images/video_images3/Animation_pump_{}.png".format(n))
    fig.show()
    U0 = U1 
    Ek0 = Ek1
    s0 = s1  
    n+=1


# Corriente por cuadrante

$$ R(\nabla \cdot J) = \frac{d\rho}{dt}R$$

$$ R \cdot R(\nabla \cdot J) = \frac{d\rho}{dt}R \cdot R$$

$$ \frac{R \cdot R}{|R|^2}(\nabla \cdot J_{cell}) = \frac{d\rho_{cell}}{dt}\frac{R \cdot R}{|R|^2}$$


$$\int J_{cell} \cdot dA = \int \frac{d\rho_{cell}}{dt} dV$$

$$ J_{cell} =  \sum_{cell} \frac{d\rho_{cell}}{dt}  $$

In [ ]:
import plotly.figure_factory as ff
Rx = []
Ry = []
for γ in linspace(0,2.5,100):
    Pi_array = linspace(-pi,pi,100)

    H0 = Hoti_Hamiltonian_transport(0.5,-pi,γ)
    Ek0, U0 = eigh(H0)
    Ek0 = np.real(Ek0)
    s0 = density_of_states(Ek0.T,U0.T)
    n = 0
    jx = []
    jy = []
    for θ in Pi_array[1:]:
        H1 = Hoti_Hamiltonian_transport(0.5,θ,γ)
        Ek1, U1 = eigh(H1)
        Ek1 = np.real(Ek1)
        s1 = density_of_states(Ek1.T,U1.T)

        Cx = [] 
        Cy = []
        Cu = [] 
        Cv = []
        for z in np.unique(f):
            find = [j for j,val in enumerate(f) if val == z]
            X = [x[i] for i in find]
            Y = [y[i] for i in find]
            S0 = [s0[i] for i in find]
            S1 = [s1[i] for i in find]
            xmax = np.amax(X)
            xmin = np.amin(X)
            ymax = np.amax(Y)
            ymin = np.amin(Y)
            rx = (xmax + xmin)/2
            ry = (ymax + ymin)/2
            if (rx < 9) and (ry < 9):
                #print("({},{})".format(rx,ry))
                Jx = vdot(subtract(S1,S0),X-rx)
                Jy = vdot(subtract(S1,S0),Y-ry)
                Cx.append(rx)
                Cy.append(ry)
                Cu.append(2000*Jx)
                Cv.append(2000*Jy)

        jx.append(sum(Cu))
        jy.append(sum(Cv))
        U0 = U1 
        Ek0 = Ek1
        s0 = s1  
        n+=1
    Rx.append(jx)
    Ry.append(jy)

savetxt("Rx.txt",Rx,fmt = '%s', comments = '')  
savetxt("Ry.txt",Ry,fmt = '%s', comments = '')  

In [ ]:
Rx  = loadtxt('Rx.txt',unpack=True) 
Rx  = asmatrix(Rx.T)
val = where(Rx == amax(Rx))[0][0]
Xs  = linspace(-pi,pi,100)[1:]
Ys  = linspace(0,2.5,100)
        
fig = go.Figure(go.Heatmap(x=Xs,y=Ys,z=Rx, colorscale="RdBu"))
#fig.add_trace(go.Scatter(x=[-pi,pi],y=[Ys[val],Ys[val]],mode='lines', marker_color="black",line=dict(width=1)))
fig.update_layout(height=600, width=600, title_text="")
fig.update_xaxes(title_text='θ')
fig.update_yaxes(title_text='γ/λ')
fig.write_image("images/param_pump_Currentx_{}.png".format(round(val,2)))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(x=linspace(-pi,pi,100)[1:],y=linspace(0,2.5,100),z=Rx, colorscale="RdBu")])

fig.update_layout(title='', autosize=False,
                  width=700, height=700,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.write_image("images/param_pump_Currentx3D_{}.png".format(round(val,2)))

fig.show()

In [ ]:
Ry = asmatrix(Ry)
val = where(Ry == amax(Ry))[0][0]

fig = go.Figure(go.Heatmap(x=linspace(-pi,pi,50)[1:],y=linspace(0,3,30),z=Ry , colorscale="RdBu"))
fig.update_layout(height=600, width=600, title_text="")
fig.update_xaxes(title_text='θ')
fig.update_yaxes(title_text='γ/λ')
fig.write_image("images/param_pump_Currenty_{}.png".format(round(val,2)))
fig.show()